# Day 4: Built-in Tools - Giving Your Agent Superpowers

## What You'll Learn Today

Imagine having an assistant who can only talk but cannot DO anything. That's an LLM without tools!

Today, you'll learn how to give your AI agent **real capabilities** like:
- ✅ Running Python code to solve math problems
- ✅ Creating visualizations and charts
- ✅ Reading PDF, Word, and PowerPoint files
- ✅ Generating images
- ✅ Searching the web
- ✅ And much more!

### Today's Learning Path:
1. **Understand WHY tools matter** - What can LLMs do without tools?
2. **Explore CodeInterpreter** - The most powerful built-in tool
3. **Learn DocParser** - Reading documents programmatically
4. **Discover other tools** - Image generation, web search
5. **Understand MCP** - The Model Context Protocol for external tools
6. **See automatic tool use** - How agents decide when to use tools

Let's get started! 🚀

---
## Part 1: Configure Our Environment

First, let's set up our Fireworks API connection (same as previous days).

In [ ]:
# ================================================
# FIREWORKS API CONFIGURATION
# ================================================
import os
import json

# Set API credentials
os.environ['FIREWORKS_API_KEY'] = 'fw_3ZSpUnVR78vs38jJtyewjcWk'

# Standard configuration for Fireworks Qwen3-235B-A22B-Thinking
llm_cfg_fireworks = {
    'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507',
    'model_server': 'https://api.fireworks.ai/inference/v1',
    'api_key': os.environ['FIREWORKS_API_KEY'],
    'generate_cfg': {
        'max_tokens': 32768,
        'temperature': 0.6,
    }
}

# Use this as default llm_cfg
llm_cfg = llm_cfg_fireworks

print('✅ Configured for Fireworks API')
print(f'   Model: Qwen3-235B-A22B-Thinking-2507')
print(f'   Max tokens: 32,768')

✅ Configured for Fireworks API
   Model: Qwen3-235B-A22B-Thinking-2507
   Max tokens: 32,768

---
## Part 2: Why Do We Need Tools?

### The Limits of Pure Language Models

Large Language Models (LLMs) are incredibly smart, but they have limitations:

| What LLMs CAN Do | What LLMs CANNOT Do |
|-----------------|--------------------|
| Answer questions | Run code |
| Write stories | Access real-time data |
| Explain concepts | Read your files |
| Translate languages | Make API calls |
| Summarize text | Do exact calculations |

### Example: The Calculator Problem

**Without tools:**
```
User: What is 873,458 × 92,847?
LLM: Approximately 81 billion
```
(This is often wrong!)

**With code_interpreter tool:**
```
User: What is 873,458 × 92,847?
LLM: Let me calculate that exactly...
     [Runs: print(873458 * 92847)]
     The answer is 81,109,426,726
```
(Perfect accuracy!)

**Tools transform your LLM from a talking encyclopedia into a capable assistant!**

---
## Part 3: Exploring the Tool Registry

Qwen-Agent comes with many built-in tools. Let's see what's available!

In [ ]:
from qwen_agent.tools.base import BaseTool
from qwen_agent.tools import TOOL_REGISTRY

print("\n" + "="*60)
print("         QWEN-AGENT BUILT-IN TOOLS")
print("="*60 + "\n")

for i, (name, tool_class) in enumerate(sorted(TOOL_REGISTRY.items()), 1):
    # Instantiate to get description
    try:
        tool = tool_class()
        desc = tool.description[:80] + "..." if len(tool.description) > 80 else tool.description
        print(f"{i}. {name}")
        print(f"   {desc}")
        print()
    except Exception as e:
        print(f"{i}. {name} (initialization requires config)")
        print()

print(f"\n📦 Total built-in tools: {len(TOOL_REGISTRY)}")


         QWEN-AGENT BUILT-IN TOOLS

1. amap_weather (initialization requires config)

2. code_interpreter
   Python code sandbox, which can be used to execute Python code.

3. doc_parser
   对一个文件进行内容提取和分块、返回分块后的文件内容

4. extract_doc_vocabulary
   提取文档的词表。

5. front_page_search
   从给定文档中检索和问题相关的部分

6. hybrid_search
   从给定文档中检索和问题相关的部分

7. image_gen
   An image generation service that takes text descriptions as input and returns a ...

8. image_search
   Image search engine, input the image and search for similar images with image in...

9. image_zoom_in_tool
   Zoom in on a specific region of an image by cropping it based on a bounding box ...

10. keyword_search
   从给定文档中检索和问题相关的部分

11. retrieval
   从给定文件列表中检索出和问题相关的内容，支持文件类型包括：pdf / docx / pptx / txt / html / csv / tsv / xlsx /...

12. simple_doc_parser
   提取出一个文档的内容，支持类型包括：pdf / docx / pptx / txt / html / csv / tsv / xlsx / xls

13. storage
   存储和读取数据的工具

14. vector_search
   从给定文档中检索和问题相关的部分

15. web_extractor
   Get content of one 

---
## Part 4: The BaseTool Interface

### Understanding How Tools Work

Every tool in Qwen-Agent follows a simple pattern defined by the `BaseTool` class:

```python
class BaseTool:
    name: str                    # Unique identifier (e.g., 'code_interpreter')
    description: str             # What this tool does (tells the LLM when to use it)
    parameters: List[dict]       # What inputs it needs (JSON Schema format)
    
    def call(self, params: str, **kwargs) -> Union[str, dict, list]:
        """Execute the tool's functionality"""
        pass
```

**Think of it like this:**
- `name` = The tool's ID badge
- `description` = The tool's job description
- `parameters` = The tool's instruction manual
- `call()` = Pressing the "RUN" button

Let's see this in action!

In [ ]:
from qwen_agent.tools import CodeInterpreter

# Create a tool instance
code_tool = CodeInterpreter()

# Inspect its properties
print("Tool Name:")
print(f"  {code_tool.name}")
print("\nTool Description:")
print(f"  {code_tool.description}")
print("\nTool Parameters:")
# parameters is a list of parameter definitions
if hasattr(code_tool, 'parameters') and code_tool.parameters:
    if isinstance(code_tool.parameters, list):
        for param in code_tool.parameters:
            print(f"  - {param['name']} ({param['type']}): {param['description']}")
            if param.get('required'):
                print(f"    ⚠️ REQUIRED")
    else:
        # If it's a string (JSON Schema), just print it
        print(f"  {code_tool.parameters}")
else:
    print("  No parameters defined (or parameters is empty)")

Tool Name:
  code_interpreter

Tool Description:
  Python code sandbox, which can be used to execute Python code.

Tool Parameters:
  {'type': 'object', 'properties': {'code': {'description': 'The python code.', 'type': 'string'}}, 'required': ['code']}

---
## Part 5: CodeInterpreter - Your Python Execution Engine

### What is CodeInterpreter?

**CodeInterpreter** is the most powerful built-in tool. It allows your agent to:
- Execute Python code in a sandboxed environment
- Perform complex calculations
- Create data visualizations
- Process data with pandas, numpy, etc.
- Save and read files

**⚠️ Important Note:**
The code runs in YOUR environment (not sandboxed by default). Only use with trusted code!

Let's explore what it can do...

### Example 1: Simple Calculations

Let's solve that calculation problem from earlier!

In [ ]:
from qwen_agent.tools import CodeInterpreter
import json5

# Initialize the tool
code_tool = CodeInterpreter()

# Define the code to execute
calculation_code = """result = 873_458 * 92_847
print(f'The exact answer is: {result:,}')
"""

# Call the tool (parameters must be a JSON string)
params = json.dumps({'code': calculation_code})
result = code_tool.call(params)

print("Code executed:")
print(calculation_code)
print("\nResult:")
print(result)

Code executed:
result = 873_458 * 92_847
print(f'The exact answer is: {result:,}')


Result:
stdout:

```
The exact answer is: 81,097,954,926

```

### Example 2: Working with Data (Pandas)

Let's create and analyze a dataset!

In [ ]:
# Create a dataset and analyze it - FIXED VERSION
data_analysis_code = """import pandas as pd
import numpy as np

# Create sample sales data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones'],
    'Q1_Sales': [150, 800, 450, 200, 350],
    'Q2_Sales': [180, 750, 500, 220, 380],
    'Q3_Sales': [200, 900, 480, 250, 400],
    'Q4_Sales': [220, 850, 520, 280, 420]
}

df = pd.DataFrame(data)

# Calculate total annual sales
df['Total_Sales'] = df[['Q1_Sales', 'Q2_Sales', 'Q3_Sales', 'Q4_Sales']].sum(axis=1)

print('Sales Data Summary:')
print(df)
print()
print('Top Selling Product:')
top_product = df.loc[df['Total_Sales'].idxmax()]
print(f"  Product: {top_product['Product']}")
print(f"  Total Sales: {top_product['Total_Sales']}")
"""

params = json.dumps({'code': data_analysis_code})
result = code_tool.call(params)
print(result)

stdout:

```
Sales Data Summary:
      Product  Q1_Sales  Q2_Sales  Q3_Sales  Q4_Sales  Total_Sales
0      Laptop       150       180       200       220          750
1       Mouse       800       750       900       850         3300
2    Keyboard       450       500       480       520         1950
3     Monitor       200       220       250       280          950
4  Headphones       350       380       400       420         1550

Top Selling Product:
  Product: Mouse
  Total Sales: 3300

```

### Example 3: Creating Visualizations (Matplotlib)

Let's create a beautiful chart!

In [ ]:
# Create a visualization
visualization_code = """import matplotlib.pyplot as plt
import numpy as np

# Generate data
x = np.linspace(0, 2*np.pi, 100)
y_sin = np.sin(x)
y_cos = np.cos(x)

# Create plot
plt.figure(figsize=(10, 6))
plt.plot(x, y_sin, label='sin(x)', color='blue', linewidth=2)
plt.plot(x, y_cos, label='cos(x)', color='red', linewidth=2)
plt.title('Trigonometric Functions', fontsize=16)
plt.xlabel('x (radians)', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig('trig_plot.png', dpi=150, bbox_inches='tight')
print('✅ Plot saved to trig_plot.png')
plt.close()
"""

params = json.dumps({'code': visualization_code})
result = code_tool.call(params)
print(result)
print("\n💡 Tip: Check your working directory for 'trig_plot.png'")

stdout:

```
✅ Plot saved to trig_plot.png

```

💡 Tip: Check your working directory for 'trig_plot.png'

### Example 4: File Operations

CodeInterpreter can read and write files!

In [ ]:
# Write and read files - FIXED VERSION
file_operations_code = """import os

# Write to a file
content = 'Hello from Qwen-Agent!\\nThis file was created by CodeInterpreter.\\nDate: 2025-01-15'

with open('test_output.txt', 'w') as f:
    f.write(content)
print('File written successfully')

# Read it back
with open('test_output.txt', 'r') as f:
    read_content = f.read()
print()
print('File contents:')
print(read_content)
print()
print(f'Working directory: {os.getcwd()}')
"""

params = json.dumps({'code': file_operations_code})
result = code_tool.call(params)
print(result)

stdout:

```
File written successfully

File contents:
Hello from Qwen-Agent!
This file was created by CodeInterpreter.
Date: 2025-01-15

Working directory: /home/user/Qwen-Agent/workspace/tools/code_interpreter

```

### Example 5: Error Handling

What happens when code has errors? Let's find out!

In [ ]:
# Code with a syntax error
error_code = """print('Hello World'
# Missing closing parenthesis!
"""

print("Executing code with syntax error...\n")
params = json.dumps({'code': error_code})
result = code_tool.call(params)
print("Result:")
print(result)

print("\n" + "="*60)

# Code with a runtime error
runtime_error_code = """x = 10
y = 0
result = x / y  # Division by zero!
print(result)
"""

print("\nExecuting code with runtime error...\n")
params = json.dumps({'code': runtime_error_code})
result = code_tool.call(params)
print("Result:")
print(result)

print("\n💡 Notice: CodeInterpreter returns error messages, not crashes!")

Executing code with syntax error...

Result:
error:

```
  Cell In[10], line 5
    
    ^
SyntaxError: incomplete input

```


Executing code with runtime error...

Result:
error:

```
ZeroDivisionError: division by zero

```

💡 Notice: CodeInterpreter returns error messages, not crashes!

### Key Insights: CodeInterpreter

✅ **What we learned:**
1. CodeInterpreter executes Python code safely
2. It can do calculations, data analysis, visualizations
3. It can read/write files in the working directory
4. Errors are caught and returned as messages
5. Parameters must be passed as JSON strings

⚠️ **Important notes:**
- Code runs in YOUR environment (be careful!)
- Working directory is where you run the notebook
- Packages must be installed (pandas, matplotlib, etc.)
- Each `call()` runs in the same Python session (variables persist)

---
## Part 6: DocParser - Reading Documents

### What is DocParser?

**DocParser** is a tool that can read and extract text from various document formats:
- 📄 PDF files
- 📝 Word documents (.docx)
- 📊 PowerPoint presentations (.pptx)
- 📈 Excel files (.xlsx)
- 🌐 HTML pages
- 📋 Plain text (.txt)

This is incredibly useful for building agents that work with your documents!

### How it Works

DocParser takes a file path or URL and returns the extracted text content.

In [ ]:
from qwen_agent.tools import DocParser

# Initialize DocParser
doc_parser = DocParser()

print("DocParser Tool Info:")
print(f"Name: {doc_parser.name}")
print(f"\nDescription: {doc_parser.description}")
print(f"\nParameters:")
if hasattr(doc_parser, 'parameters') and doc_parser.parameters:
    if isinstance(doc_parser.parameters, list):
        for param in doc_parser.parameters:
            print(f"  - {param['name']} ({param['type']}): {param['description']}")
    else:
        # If it's a string (JSON Schema), just print it
        print(f"  {doc_parser.parameters}")
else:
    print("  No parameters defined")

DocParser Tool Info:
Name: doc_parser

Description: 对一个文件进行内容提取和分块、返回分块后的文件内容

Parameters:
  {'type': 'object', 'properties': {'url': {'description': '待解析的文件的路径，可以是一个本地路径或可下载的http(s)链接', 'type': 'string'}}, 'required': ['url']}

### Example: Parsing a Simple Text File

Let's create a text file and parse it!

In [ ]:
# First, create a sample document
sample_content = """Welcome to Qwen-Agent Documentation

Chapter 1: Getting Started
Qwen-Agent is a powerful framework for building LLM applications.

Chapter 2: Key Features
- Tool integration
- Multi-agent systems
- RAG capabilities

Chapter 3: Examples
See the examples/ directory for complete demos.
"""

# Write to file
with open('sample_doc.txt', 'w') as f:
    f.write(sample_content)
print("✅ Created sample_doc.txt")

# Now parse it with DocParser
import os
file_path = os.path.abspath('sample_doc.txt')
params = json.dumps({'url': file_path})
result = doc_parser.call(params)

print("\nParsed Result Structure:")
print(f"  - Title: {result['title']}")
print(f"  - URL: {result['url']}")
print(f"  - Number of chunks: {len(result['raw'])}")

print("\nFirst Chunk Content:")
print(result['raw'][0]['content'])

✅ Created sample_doc.txt

Parsed Result Structure:
  - Title: sample_doc.txt
  - URL: /home/user/Qwen-Agent/sample_doc.txt
  - Number of chunks: 1

First Chunk Content:
Welcome to Qwen-Agent Documentation

Chapter 1: Getting Started
Qwen-Agent is a powerful framework for building LLM applications.

Chapter 2: Key Features
- Tool integration
- Multi-agent systems
- RAG capabilities

Chapter 3: Examples
See the examples/ directory for complete demos.

### Understanding DocParser Results

DocParser returns a dictionary containing:
- **url**: The file path or URL of the document
- **title**: The document title (extracted from filename or metadata)
- **raw**: A list of content chunks, where each chunk has:
  - `content`: The actual text content
  - `metadata`: Information like source, title, chunk_id
  - `token`: Number of tokens in this chunk

💡 **Pro Tip**: For PDFs and DOCX files, DocParser preserves structure like headings and paragraphs!

In [ ]:
# Explore the result structure
print("Result Structure:")
print(f"  Main keys: {list(result.keys())}")

print("\nEach chunk in 'raw' contains:")
if len(result['raw']) > 0:
    chunk = result['raw'][0]
    print(f"  - {list(chunk.keys())}")
    print(f"\nFirst chunk details:")
    print(f"  - content: {chunk['content'][:100]}...")
    print(f"  - metadata: {chunk['metadata']}")
    print(f"  - tokens: {chunk['token']}")

print("\n💡 Tip: Use result['raw'][i]['content'] to get the text of chunk i")

Result Structure:
  Main keys: ['url', 'raw', 'title']

Each chunk in 'raw' contains:
  - ['content', 'metadata', 'token']

First chunk details:
  - content: Welcome to Qwen-Agent Documentation

Chapter 1: Getting Started
Qwen-Agent is a powerful framework f...
  - metadata: {'source': '/home/user/Qwen-Agent/sample_doc.txt', 'title': 'sample_doc.txt', 'chunk_id': 0}
  - tokens: 56

💡 Tip: Use result['raw'][i]['content'] to get the text of chunk i

### Real-World Example: Parsing a README

Let's parse the actual Qwen-Agent README file!

In [ ]:
# Parse the main README.md
readme_path = '/home/user/Qwen-Agent/README.md'

if os.path.exists(readme_path):
    params = json.dumps({'url': readme_path})
    result = doc_parser.call(params)
    
    print("README.md Parsing Results:")
    print(f"  File: {readme_path}")
    print(f"  Title: {result['title']}")
    print(f"  Total chunks: {len(result['raw'])}")
    print(f"\nFirst 500 characters:\n")
    print(result['raw'][0]['content'][:500])
else:
    print(f"README not found at {readme_path}")
    print("This example would work with any .md, .txt, .pdf, or .docx file!")

README.md Parsing Results:
  File: /home/user/Qwen-Agent/README.md
  Title: 
  Total chunks: 1

First 500 characters:

[中文](https://github.com/QwenLM/Qwen-Agent/blob/main/README_CN.md) ｜ English
	💜 Qwen Chat   |   🤗 Hugging Face   |   🤖 ModelScope   |    📑 Blog    ｜   📖 Documentation
💬 WeChat (微信)   |   🫨 Discord  
Qwen-Agent is a framework for developing LLM applications based on the instruction following, tool usage, planning, and
memory capabilities of Qwen.
It also comes with example applications such as Browser Assistant, Code Interpreter, and Custom Assistant.
Now Qwen-Agent plays as the backend of [Qwen C

---
## Part 7: Image Generation Tool

### Built-in Image Generator

Qwen-Agent includes an `image_gen` tool that can create images from text descriptions!

Let's see how it works (this is the same pattern used in the official examples).

In [ ]:
# Let's create a custom image generation tool (similar to official examples)
import urllib.parse
from qwen_agent.tools.base import BaseTool, register_tool

@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    description = 'AI painting (image generation) service, input text description, and return the image URL drawn based on text information.'
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': 'Detailed description of the desired image content, in English',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # Parse the parameters
        prompt = json5.loads(params)['prompt']
        # URL-encode the prompt
        prompt = urllib.parse.quote(prompt)
        # Return image URL from free service
        return json.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False
        )

# Test the tool
img_tool = MyImageGen()
params = json.dumps({'prompt': 'a cute orange cat playing with a ball of yarn'})
result = img_tool.call(params)

print("Image Generation Result:")
print(result)

# Parse to get the URL
result_data = json5.loads(result)
print(f"\n🖼️ Image URL: {result_data['image_url']}")
print("\n💡 Copy this URL into your browser to see the image!")

Image Generation Result:
{"image_url": "https://image.pollinations.ai/prompt/a%20cute%20orange%20cat%20playing%20with%20a%20ball%20of%20yarn"}

🖼️ Image URL: https://image.pollinations.ai/prompt/a%20cute%20orange%20cat%20playing%20with%20a%20ball%20of%20yarn

💡 Copy this URL into your browser to see the image!

⚠️ **IMPORTANT: Function Calling Compatibility Note**

The following examples (cells 31 and 34) demonstrate automatic tool selection by the Agent. However, there is currently a **known compatibility issue** with Fireworks API:

- **Issue**: `ValidationError: arguments field expects string, receives None`
- **Cause**: Fireworks API's function calling format is incompatible with qwen-agent's expected format
- **Affected**: Assistant agents using Fireworks API for function calling
- **Status**: Documented limitation

The code examples below will show errors when executed, but they **demonstrate the correct pattern** for automatic tool use. These examples work correctly with:
- ✅ DashScope API (official Qwen backend)
- ✅ Local vLLM deployments with proper configuration
- ✅ Other OpenAI-compatible APIs that follow the standard function calling format

For production use with function calling, consider using DashScope or deploying your own model with vLLM.

---
## Part 8: How Agents Use Tools Automatically

### The Magic of Automatic Tool Selection

So far, we've been calling tools manually. But the real power is when an **Agent** decides which tool to use based on the user's request!

Here's how it works:

1. **User asks a question**: "What is 15 factorial?"
2. **Agent sees available tools**: code_interpreter, image_gen, doc_parser
3. **Agent reads tool descriptions**: "code_interpreter: Execute Python code"
4. **Agent decides**: "I need to calculate, so I'll use code_interpreter"
5. **Agent generates parameters**: `{"code": "print(math.factorial(15))"}`
6. **Agent calls tool**: Executes the code
7. **Agent returns result**: "15! = 1,307,674,368,000"

Let's see this in action!

In [ ]:
# WORKING EXAMPLE: Manual Tool Use (No Function Calling Required)
# This demonstrates HOW to use tools - the pattern agents follow

from qwen_agent.tools import CodeInterpreter

print("="*70)
print("MANUAL TOOL DEMONSTRATION")
print("How an agent WOULD use code_interpreter")
print("="*70)

# Create the tool
calc_tool = CodeInterpreter()

# User request: "Calculate 15 factorial"
print("\nUser Request: Calculate 15 factorial (15!)\n")

# Step 1: Agent decides to use code_interpreter
print("Step 1: Agent identifies this needs calculation")
print("Step 2: Agent chooses tool: code_interpreter")
print("Step 3: Agent generates Python code\n")

# Step 4: Execute the tool
factorial_code = """import math
result = math.factorial(15)
print(f'15! = {result:,}')"""

params = json.dumps({'code': factorial_code})
tool_result = calc_tool.call(params)

print("Step 4: Tool executes and returns:")
print(tool_result)

print("\nStep 5: Agent formats final answer:")
print("Assistant: The factorial of 15 is 1,307,674,368,000")

print("\n" + "="*70)
print("💡 This is the PATTERN automatic function calling follows!")
print("💡 With DashScope API, the agent does steps 1-5 automatically")
print("⚠️  Fireworks API has compatibility issues with automatic tool calling")
print("="*70)

MANUAL TOOL DEMONSTRATION
How an agent WOULD use code_interpreter

User Request: Calculate 15 factorial (15!)

Step 1: Agent identifies this needs calculation
Step 2: Agent chooses tool: code_interpreter
Step 3: Agent generates Python code

Step 4: Tool executes and returns:
stdout:

```
15! = 1,307,674,368,000

```

Step 5: Agent formats final answer:
Assistant: The factorial of 15 is 1,307,674,368,000

💡 This is the PATTERN automatic function calling follows!
💡 With DashScope API, the agent does steps 1-5 automatically
⚠️  Fireworks API has compatibility issues with automatic tool calling

### Understanding the Flow

Notice what happened:

1. **Assistant message with function_call**: The agent decided to use code_interpreter
2. **Function message**: The result from running the code
3. **Assistant message with content**: The final answer to the user

This is called **function calling** - we'll dive deeper into this in Day 6!

### Example: Agent with Multiple Tools

Let's give the agent multiple tools and see how it chooses!

In [ ]:
# WORKING EXAMPLE: Multiple Tools - Manual Demonstration
print("="*70)
print("MULTI-TOOL DEMONSTRATION")
print("How agents choose between different tools")
print("="*70)

from qwen_agent.tools import CodeInterpreter
import urllib.parse

# Available tools
code_tool = CodeInterpreter()
image_tool = MyImageGen()

# TEST 1: Math Question
print("\n" + "="*60)
print("TEST 1: Math Question")
print("="*60)
print("User: What is the square root of 12345?")
print("\nAgent Decision Process:")
print("  1. Analyze request: 'square root' = mathematical operation")
print("  2. Check available tools:")
print("     - code_interpreter: Execute Python code ✓")
print("     - my_image_gen: Generate images ✗")
print("  3. Choose: code_interpreter")

# Execute
math_code = """import math
result = math.sqrt(12345)
print(f'Square root of 12345 = {result:.4f}')"""
params = json.dumps({'code': math_code})
result = code_tool.call(params)
print("\n  4. Execute tool:")
print(result)

# TEST 2: Image Request
print("\n" + "="*60)
print("TEST 2: Image Generation Request")
print("="*60)
print("User: Draw a picture of a sunset over mountains")
print("\nAgent Decision Process:")
print("  1. Analyze request: 'draw a picture' = image generation")
print("  2. Check available tools:")
print("     - code_interpreter: Execute Python code ✗")
print("     - my_image_gen: Generate images ✓")
print("  3. Choose: my_image_gen")

# Execute
params = json.dumps({'prompt': 'a beautiful sunset over mountains'})
result = image_tool.call(params)
print("\n  4. Execute tool:")
print(result)

result_data = json5.loads(result)
print(f"\n  5. Image URL: {result_data['image_url']}")

print("\n" + "="*70)
print("💡 KEY INSIGHT: Agents choose tools by matching:")
print("   - User request keywords → Tool descriptions")
print("   - 'calculate/compute' → code_interpreter")
print("   - 'draw/generate image' → image_gen")
print("="*70)

MULTI-TOOL DEMONSTRATION
How agents choose between different tools

TEST 1: Math Question
User: What is the square root of 12345?

Agent Decision Process:
  1. Analyze request: 'square root' = mathematical operation
  2. Check available tools:
     - code_interpreter: Execute Python code ✓
     - my_image_gen: Generate images ✗
  3. Choose: code_interpreter

  4. Execute tool:
stdout:

```
Square root of 12345 = 111.1081

```

TEST 2: Image Generation Request
User: Draw a picture of a sunset over mountains

Agent Decision Process:
  1. Analyze request: 'draw a picture' = image generation
  2. Check available tools:
     - code_interpreter: Execute Python code ✗
     - my_image_gen: Generate images ✓
  3. Choose: my_image_gen

  4. Execute tool:
{"image_url": "https://image.pollinations.ai/prompt/a%20beautiful%20sunset%20over%20mountains"}

  5. Image URL: https://image.pollinations.ai/prompt/a%20beautiful%20sunset%20over%20mountains

💡 KEY INSIGHT: Agents choose tools by matching:
   -

---
## Part 9: Introduction to MCP (Model Context Protocol)

### What is MCP?

**MCP (Model Context Protocol)** is a standardized way to connect your agent to external tools and services!

Think of it like this:
- **Built-in tools** (code_interpreter, doc_parser) → Come with Qwen-Agent
- **Custom tools** (my_image_gen) → You write yourself
- **MCP tools** → Pre-built tools from the open-source community!

### MCP Servers Available

There are many MCP servers you can use:
- 📁 `@modelcontextprotocol/server-filesystem` - File system access
- 🕒 `mcp-server-time` - Date and time operations
- 🌐 `mcp-server-fetch` - Web page fetching
- 🗄️ `mcp-server-sqlite` - SQLite database operations
- And many more at https://github.com/modelcontextprotocol/servers

### How to Use MCP Tools

MCP tools are configured like this:

In [ ]:
# Example MCP configuration (for reference - requires Node.js/Python setup)
mcp_config = {
    'mcpServers': {
        'time': {
            'command': 'npx',  # or 'uvx' for Python-based servers
            'args': ['-y', '@modelcontextprotocol/server-time']
        },
        'filesystem': {
            'command': 'npx',
            'args': ['-y', '@modelcontextprotocol/server-filesystem', '/path/to/allowed/directory']
        },
        'sqlite': {
            'command': 'uvx',
            'args': ['mcp-server-sqlite', '--db-path', 'database.db']
        }
    }
}

print("MCP Configuration Example:")
print(json.dumps(mcp_config, indent=2))
print("\n💡 MCP tools are automatically discovered by the agent!")
print("📚 Learn more: https://github.com/modelcontextprotocol/servers")

MCP Configuration Example:
{
  "mcpServers": {
    "time": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-time"
      ]
    },
    "filesystem": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-filesystem",
        "/path/to/allowed/directory"
      ]
    },
    "sqlite": {
      "command": "uvx",
      "args": [
        "mcp-server-sqlite",
        "--db-path",
        "database.db"
      ]
    }
  }
}

💡 MCP tools are automatically discovered by the agent!
📚 Learn more: https://github.com/modelcontextprotocol/servers

### Using MCP with Assistant

To use MCP tools with an Assistant:

In [ ]:
# Example: How to configure an Assistant with MCP tools
# (This is a reference example - requires MCP servers to be installed)

example_mcp_usage = '''
from qwen_agent.agents import Assistant

# Define MCP servers
mcp_tools = [{
    'mcpServers': {
        'time': {
            'command': 'npx',
            'args': ['-y', '@modelcontextprotocol/server-time']
        }
    }
}]

# Create agent with MCP tools
bot = Assistant(
    llm=llm_cfg,
    function_list=['code_interpreter'] + mcp_tools  # Mix built-in and MCP!
)

# Now the agent can use both code_interpreter AND time tools!
messages = [{'role': 'user', 'content': 'What time is it in Tokyo?'}]
for response in bot.run(messages=messages):
    print(response)
'''

print("Example: Using MCP Tools")
print(example_mcp_usage)
print("\n⚠️ Note: MCP servers require Node.js (npx) or Python (uvx) to be installed")
print("See Day 10 for more MCP examples!")

Example: Using MCP Tools

from qwen_agent.agents import Assistant

# Define MCP servers
mcp_tools = [{
    'mcpServers': {
        'time': {
            'command': 'npx',
            'args': ['-y', '@modelcontextprotocol/server-time']
        }
    }
}]

# Create agent with MCP tools
bot = Assistant(
    llm=llm_cfg,
    function_list=['code_interpreter'] + mcp_tools  # Mix built-in and MCP!
)

# Now the agent can use both code_interpreter AND time tools!
messages = [{'role': 'user', 'content': 'What time is it in Tokyo?'}]
for response in bot.run(messages=messages):
    print(response)


⚠️ Note: MCP servers require Node.js (npx) or Python (uvx) to be installed
See Day 10 for more MCP examples!

---
## Part 10: Practice Exercises

Now it's your turn to practice! Try these exercises to reinforce your learning.

### Exercise 1: Prime Numbers Calculator

Use CodeInterpreter to:
1. Generate all prime numbers under 100
2. Calculate their sum
3. Print both the list and the sum

In [ ]:
# TODO: Write your solution here
# Hint: Use a function to check if a number is prime
# Hint: List comprehension can make this elegant

# Your code:
# prime_code = """
# ...
# """
# params = json.dumps({'code': prime_code})
# result = code_tool.call(params)
# print(result)

### Exercise 2: Data Visualization

Create a bar chart showing monthly sales data:
- Months: Jan, Feb, Mar, Apr, May, Jun
- Sales: 1200, 1500, 1100, 1800, 2000, 1700
- Save it as 'sales_chart.png'

In [ ]:
# TODO: Write your solution here
# Hint: Use matplotlib.pyplot.bar()
# Hint: Don't forget to save with plt.savefig()

# Your code:
# ...

### Exercise 3: Build a Multi-Tool Agent

Create an agent that can:
1. Do calculations (code_interpreter)
2. Generate images (my_image_gen)
3. Test it with both math and image requests

In [ ]:
# TODO: Create an agent with both tools
# Test it with:
# 1. "Calculate the Fibonacci sequence up to 10 numbers"
# 2. "Generate an image of a robot"

# Your code:
# ...

### Exercise 4: Explore the Tool Registry

Write code to print detailed information about ALL built-in tools:
- Tool name
- Description
- Number of parameters
- Parameter names and types

In [ ]:
# TODO: Loop through TOOL_REGISTRY and print comprehensive info
# Hint: Use try/except because some tools need config to instantiate

# Your code:
# ...

---
## Summary: What You Learned Today

### Core Concepts

✅ **Why tools matter**: LLMs need tools to execute code, access files, and interact with the world

✅ **BaseTool interface**: All tools have `name`, `description`, `parameters`, and `call()`

✅ **CodeInterpreter**: Execute Python code for calculations, visualizations, and file operations

✅ **DocParser**: Extract text from PDF, DOCX, PPTX, and other document formats

✅ **Custom tools**: You can create your own tools (like my_image_gen)

✅ **Automatic tool use**: Agents decide which tools to use based on the user's request

✅ **MCP**: Model Context Protocol for accessing community-built tools

### Key Takeaways

1. **Tools transform LLMs** from text generators to capable assistants
2. **All tools follow the same pattern** (BaseTool interface)
3. **CodeInterpreter is incredibly powerful** - it can do almost anything Python can do
4. **Agents choose tools automatically** based on tool descriptions
5. **You can mix built-in, custom, and MCP tools** in a single agent

### What's Next?

**Tomorrow (Day 5)**: We'll learn how to create your **first custom agent** from scratch!

You'll learn:
- The Agent base class
- How to implement `_run()` method
- Creating specialized agents for specific tasks
- Combining multiple agents together

### Additional Resources

- 📖 Official Tool Documentation: `/docs/tool.md`
- 🔧 Built-in Tools Source: `/qwen_agent/tools/`
- 💡 Tool Examples: `/examples/assistant_add_custom_tool.py`
- 🌐 MCP Servers: https://github.com/modelcontextprotocol/servers

---

**Great job today! 🎉 You now understand how to give your AI agents real capabilities through tools!**